In [6]:
import os
import cv2

# Utility Functions

## 1. Video Properties

In [11]:
def print_video_properties(video_input_path):
    # Capture the video from the input path
    cap = cv2.VideoCapture(video_input_path)
    
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_input_path}")
        return
    
    # Get video properties
    properties = {
        "Frame Width": cap.get(cv2.CAP_PROP_FRAME_WIDTH),
        "Frame Height": cap.get(cv2.CAP_PROP_FRAME_HEIGHT),
        "Frame Count": cap.get(cv2.CAP_PROP_FRAME_COUNT),
        "FPS": cap.get(cv2.CAP_PROP_FPS),
        "Duration (seconds)": cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS),
        "FourCC Codec": int(cap.get(cv2.CAP_PROP_FOURCC)),
        "Brightness": cap.get(cv2.CAP_PROP_BRIGHTNESS),
        "Contrast": cap.get(cv2.CAP_PROP_CONTRAST),
        "Saturation": cap.get(cv2.CAP_PROP_SATURATION),
        "Hue": cap.get(cv2.CAP_PROP_HUE),
        "Gain": cap.get(cv2.CAP_PROP_GAIN),
        "Exposure": cap.get(cv2.CAP_PROP_EXPOSURE),
        "Convert RGB": cap.get(cv2.CAP_PROP_CONVERT_RGB),
    }
    
    # Print video properties
    for prop, value in properties.items():
        if prop == "FourCC Codec":
            # Decode FourCC codec
            codec = (
                chr((value & 0XFF)),
                chr((value & 0XFF00) >> 8),
                chr((value & 0XFF0000) >> 16),
                chr((value & 0XFF000000) >> 24)
            )
            value = ''.join(codec)
        print(f"{prop}: {value}")
    
    # Release the video capture object
    cap.release()

In [12]:
print_video_properties(video_input_path='/home/azureuser/nikhil20.sharma/data/8972_C9.mp4')

Frame Width: 704.0
Frame Height: 480.0
Frame Count: 43162.0
FPS: 8.0
Duration (seconds): 5395.25
FourCC Codec: h264
Brightness: 0.0
Contrast: 0.0
Saturation: 0.0
Hue: 0.0
Gain: 0.0
Exposure: 0.0
Convert RGB: 1.0


## 2. Split video in chunks

In [16]:
def split_video(video_input_path, video_output_dir='output_chunks', chunk_in_min=2, no_of_chunks=None):
    # Ensure the output directory exists
    os.makedirs(video_output_dir, exist_ok=True)
    
    # Capture the video from the input path
    cap = cv2.VideoCapture(video_input_path)
    
    # Get the video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_in_sec = frame_count / fps
    chunk_in_sec = chunk_in_min * 60

    # Calculate the number of chunks
    num_chunks = int(duration_in_sec // chunk_in_sec) + 1

    # Get the video width and height
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    counter = 0
    for chunk_index in range(num_chunks):
        # Define the codec and create VideoWriter object
        output_path = os.path.join(video_output_dir, f"chunk_{chunk_index + 1}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        # Write frames for this chunk
        for frame_index in range(int(chunk_in_sec * fps)):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
        
        out.release()

        counter += 1

        if no_of_chunks is not None and counter == no_of_chunks:
            break

    
    cap.release()
    print("Video has been split into chunks and saved to:", video_output_dir)

In [18]:
# Example usage: split_video(video_input_path, video_output_dir, chunk_in_min)
split_video(
    video_input_path='/home/azureuser/nikhil20.sharma/data/8972_C9.mp4', 
    video_output_dir='/home/azureuser/nikhil20.sharma/data/8972_C9_2m_chuncks', 
    chunk_in_min=2,  # Split video into n-minute chunks
    no_of_chunks=10
)  

Video has been split into chunks and saved to: /home/azureuser/nikhil20.sharma/data/8972_C9_2m_chuncks


In [19]:
print_video_properties(video_input_path='/home/azureuser/nikhil20.sharma/data/8972_C9_2m_chuncks/chunk_1.mp4')

Frame Width: 704.0
Frame Height: 480.0
Frame Count: 960.0
FPS: 8.0
Duration (seconds): 120.0
FourCC Codec: FMP4
Brightness: 0.0
Contrast: 0.0
Saturation: 0.0
Hue: 0.0
Gain: 0.0
Exposure: 0.0
Convert RGB: 1.0


## 3. Extract Frames

In [4]:
def extract_frames(video_path, output_folder):
    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not video.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"FPS: {fps}")
    print(f"Total frames: {frame_count}")
    
    # Read frames and save as images
    frame_number = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        # Generate output file path
        output_path = os.path.join(output_folder, f"frame_{frame_number:06d}.jpg")
        
        # Save frame as image
        cv2.imwrite(output_path, frame)
        
        frame_number += 1
        
        # Print progress
        if frame_number % 100 == 0:
            print(f"Extracted {frame_number} frames")
    
    # Release video capture object
    video.release()
    
    print(f"Extraction complete. {frame_number} frames extracted.")

In [7]:
# Example usage: extract_frames(video_path, output_folder)
video_path = "../data/8972_C9_2m_chuncks/chunk_1.mp4"
output_folder = "../data/8972_C9_2m_chunck_1_frames"
extract_frames(video_path, output_folder)

Error: Could not open video.


OpenCV: Couldn't read video stream from file "../data/8972_C9_2m_chuncks/chunk_1.mp4"


# End

In [3]:
import ffmpeg
from pprint import pprint

def print_full_metadata(video_path):
    try:
        # Get the metadata of the video file
        probe = ffmpeg.probe(video_path)
        
        # Print the full metadata
        pprint(probe)

    except ffmpeg.Error as e:
        print(f"An error occurred: {e.stderr.decode()}")

# Example usage
video_path = './Data/8972/8972_C4.mp4'
print_full_metadata(video_path)

{'format': {'bit_rate': '1106617',
            'duration': '3883.896000',
            'filename': './Data/8972/8972_C4.mp4',
            'format_long_name': 'MPEG-PS (MPEG-2 Program Stream)',
            'format_name': 'mpeg',
            'nb_programs': 0,
            'nb_stream_groups': 0,
            'nb_streams': 1,
            'probe_score': 52,
            'size': '537248372',
            'start_time': '80418.716000'},
 'streams': [{'avg_frame_rate': '8/1',
              'bits_per_raw_sample': '8',
              'chroma_location': 'left',
              'closed_captions': 0,
              'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10',
              'codec_name': 'h264',
              'codec_tag': '0x0000',
              'codec_tag_string': '[0][0][0][0]',
              'codec_type': 'video',
              'coded_height': 480,
              'coded_width': 704,
              'color_primaries': 'bt709',
              'color_range': 'pc',
              'color_space': '

In [14]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Run inference on 'bus.jpg' with arguments
model.predict( 
    source='Data/8972_C9_2m_chuncks/chunk_1.mp4',
    save=True, 
    # imgsz=320, 
    conf=0.5
)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/detect/predict


[]